## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
prefix_path = '../data/mimiciv/2.0/'

## Split CHARTEVENTS to several dataset

In [9]:
count = 1
for reader in tqdm(pd.read_csv(prefix_path + 'chartevents.csv', chunksize = 30000000)):
    reader.to_csv(prefix_path + 'chartevent' + str(count) + '.csv')
    count += 1

11it [12:02, 65.71s/it]


## Load Data

In [3]:
paths = sorted(os.listdir(prefix_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('procedureevents.csv')
paths.remove('datetimeevents.csv')
paths.remove('d_hcpcs.csv')
paths.remove('transfers.csv')
paths.remove('procedures_icd.csv')
paths.remove('emar_detail.csv')
paths.remove('emar.csv')
paths.remove('services.csv')
paths.remove('microbiologyevents.csv')
paths.remove('d_icd_procedures.csv')
paths.remove('pharmacy.csv')
paths.remove('omr.csv')
paths.remove('poe.csv')
paths.remove('poe_detail.csv')
paths.remove('hcpcsevents.csv')
paths.remove('ingredientevents.csv')
paths.remove('drgcodes.csv')
paths

['admissions.csv',
 'chartevent1.csv',
 'chartevent10.csv',
 'chartevent11.csv',
 'chartevent2.csv',
 'chartevent3.csv',
 'chartevent4.csv',
 'chartevent5.csv',
 'chartevent6.csv',
 'chartevent7.csv',
 'chartevent8.csv',
 'chartevent9.csv',
 'd_icd_diagnoses.csv',
 'd_items.csv',
 'd_labitems.csv',
 'diagnoses_icd.csv',
 'icustays.csv',
 'inputevents.csv',
 'labevents.csv',
 'outputevents.csv',
 'patients.csv',
 'prescriptions.csv']

In [4]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(prefix_path + path))

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍       | 21/22 [05:36<00:18, 18.02s/it]/tmp/ipykernel_32732/136570088.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [05:57<00:00, 16.26s/it]


## Clean Data to remain SEPSIS patient and sort Data

In [8]:
i = paths.index('d_icd_diagnoses.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['long_title']]].index, inplace=True)
# D_ICD_DIAGNOSES.to_csv(prefix_path + paths[i])
D_ICD_DIAGNOSES

,icd_code,icd_version,long_title
7263,67020,9,"Puerperal sepsis, unspecified as to episode of..."
7264,67022,9,"Puerperal sepsis, delivered, with mention of p..."
7265,67024,9,"Puerperal sepsis, postpartum condition or comp..."
9163,77181,9,Septicemia [sepsis] of newborn
12078,99591,9,Sepsis
...,...,...,...
42994,R6521,10,Severe sepsis with septic shock
94949,T8144,10,Sepsis following a procedure
94950,T8144XA,10,"Sepsis following a procedure, initial encounter"
94951,T8144XD,10,"Sepsis following a procedure, subsequent encou..."


In [9]:
i = paths.index('diagnoses_icd.csv')
DIAGNOSES_ICD = datas[i].copy()
DIAGNOSES_ICD.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [10]:
i = paths.index('diagnoses_icd.csv')
datas[i].sort_values(by=['subject_id', 'seq_num'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
'''
drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['icd_code']:
        if str(DIAGNOSES_ICD.loc[j]['icd_code']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv(prefix_path + paths[i])
'''
DIAGNOSES_ICD

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5006884/5006884 [2:41:44<00:00, 515.95it/s]


,subject_id,hadm_id,seq_num,icd_code,icd_version
135,10000826,21086876,2,99591,9
537,10001401,27012892,2,A4181,10
553,10001401,27060146,2,A419,10
538,10001401,27012892,3,R6520,10
542,10001401,27012892,7,T8144XA,10
...,...,...,...,...,...
5002755,19992202,20329411,16,99592,9
5005349,19997367,20617667,4,99592,9
5005416,19997367,21508795,18,99592,9
5005780,19997886,20793010,4,A419,10


In [34]:
'''
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['subject_id'][i]].append(dataset['hadm_id'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('diagnoses_icd.csv'):
        continue
    if 'subject_id' in datas[i].columns and 'hadm_id' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['hadm_id'][j] not in dict1[datas[i]['subject_id'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        # datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'subject_id' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['subject_id'].values for id in datas[i]['subject_id']]]
        # datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)

    datas[i].to_csv(prefix_path + paths[i])
    
del dict1


i = paths.index('d_items.csv')
datas[i].sort_values(by=['itemid'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])

i = paths.index('d_labitems.csv')
datas[i].sort_values(by=['itemid'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])
'''

for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'time' in column or 'date' in column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [03:09<00:00,  8.61s/it]


# Look Data characteristics

In [35]:
i = paths.index('icustays.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
13,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222
14,10002428,28662225,33987268,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-12 16:24:18,2156-04-17 15:57:08,4.981134
15,10002428,28662225,38875437,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894
24,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213
27,10003400,23559586,34577403,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-08-10 19:54:51,2137-08-13 17:54:54,2.916701


In [36]:
i = paths.index('patients.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
25,10000826,F,32,1970-01-01 00:00:00.000002146,2008 - 2010,NaN
46,10001401,F,89,1970-01-01 00:00:00.000002131,2014 - 2016,NaN
67,10002013,F,53,1970-01-01 00:00:00.000002156,2008 - 2010,NaN
79,10002428,F,80,1970-01-01 00:00:00.000002155,2011 - 2013,NaN
111,10003019,M,69,1970-01-01 00:00:00.000002171,2008 - 2010,NaN


In [37]:
i = paths.index('admissions.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
18,10000826,21086876,2146-12-18 17:39:00,2146-12-24 19:55:00,NaT,DIRECT EMER.,CLINIC REFERRAL,HOME,Other,ENGLISH,SINGLE,WHITE,NaT,NaT,0
55,10001401,27012892,2133-07-09 22:22:00,2133-07-13 18:43:00,NaT,OBSERVATION ADMIT,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0
56,10001401,27060146,2131-10-01 01:33:00,2131-10-05 15:45:00,NaT,OBSERVATION ADMIT,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2131-09-30 16:47:00,2131-10-01 02:24:00,0
96,10002013,24848509,2162-07-08 00:08:00,2162-07-09 18:06:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,2162-07-07 19:40:00,2162-07-08 01:45:00,0
97,10002013,25442395,2166-04-06 18:47:00,2166-04-19 16:38:00,NaT,OBSERVATION ADMIT,CLINIC REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2166-04-06 16:01:00,2166-04-06 21:21:00,0


In [38]:
i = paths.index('inputevents.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
705,10002428,23473524,35479615,2156-05-11 16:30:00,2156-05-11 21:51:00,2156-05-11 17:43:00,221906,1.554480,mg,0.100054,...,Main order parameter,Continuous Med,48.4,250.0,ml,0,0,ChangeDose/Rate,8.0,0.1000
706,10002428,23473524,35479615,2156-05-11 16:30:00,2156-05-11 21:51:00,2156-05-11 17:43:00,225158,48.577483,ml,9.079904,...,Mixed solution,Continuous Med,48.4,250.0,ml,0,0,ChangeDose/Rate,250.0,9.0744
707,10002428,23473524,35479615,2156-05-11 17:15:00,2156-05-11 17:16:00,2156-05-11 17:43:00,220949,200.000000,ml,NaN,...,Mixed solution,Drug Push,48.4,200.0,ml,0,0,FinishedRunning,200.0,0.0000
708,10002428,23473524,35479615,2156-05-11 17:15:00,2156-05-11 17:16:00,2156-05-11 17:43:00,225859,1.000000,dose,NaN,...,Main order parameter,Drug Push,48.4,200.0,ml,0,0,FinishedRunning,1.0,1.0000
709,10002428,23473524,35479615,2156-05-11 18:59:00,2156-05-11 19:00:00,2156-05-11 18:59:00,220949,50.000000,ml,NaN,...,Mixed solution,Drug Push,48.4,50.0,ml,0,0,FinishedRunning,50.0,0.0000


In [39]:
i = paths.index('outputevents.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valueuom
483,10002428,23473524,35479615,2156-05-11 17:44:00,2156-05-11 17:45:00,226559,350.0,ml
484,10002428,23473524,35479615,2156-05-11 20:00:00,2156-05-11 19:49:00,226559,45.0,ml
485,10002428,23473524,35479615,2156-05-11 21:00:00,2156-05-11 22:14:00,226559,55.0,ml
486,10002428,23473524,35479615,2156-05-11 22:00:00,2156-05-11 22:14:00,226559,75.0,ml
487,10002428,23473524,35479615,2156-05-11 23:00:00,2156-05-12 00:16:00,226559,100.0,ml


In [40]:
i = paths.index('d_items.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [41]:
i = paths.index('d_labitems.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas


In [42]:
i = paths.index('labevents.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
3045,3051,10000826,21086876.0,45348866,51116,2146-12-18 18:33:00,2146-12-18 20:38:00,54,54.0,%,0.0,0.0,abnormal,STAT,NaN
3046,3052,10000826,21086876.0,45348866,51117,2146-12-18 18:33:00,2146-12-18 20:38:00,16,16.0,%,0.0,0.0,abnormal,STAT,NaN
3047,3053,10000826,21086876.0,45348866,51118,2146-12-18 18:33:00,2146-12-18 20:38:00,1,1.0,%,0.0,0.0,abnormal,STAT,NaN
3048,3054,10000826,21086876.0,45348866,51120,2146-12-18 18:33:00,2146-12-18 20:38:00,0,0.0,%,0.0,0.0,NaN,STAT,NaN
3049,3055,10000826,21086876.0,45348866,51125,2146-12-18 18:33:00,2146-12-18 20:38:00,29,29.0,%,0.0,0.0,abnormal,STAT,NaN


In [43]:
i = paths.index('prescriptions.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
286,10000826,21086876,2200640,10000826-172,172.0,2146-12-18 21:00:00,2146-12-19 20:00:00,MAIN,Albumin 25% (12.5g / 50mL),ALBU25,006329,6.746706e+10,25%-50mL,NaN,50,g,4,BAG,1.0,IV
287,10000826,21086876,5536597,10000826-222,222.0,2146-12-20 16:00:00,2146-12-21 15:00:00,MAIN,Albumin 25% (12.5g / 50mL),ALBU25,006329,6.746706e+10,25%-50mL,NaN,25,g,2,BAG,1.0,IV
288,10000826,21086876,15358102,10000826-178,178.0,2146-12-18 21:00:00,2146-12-19 21:00:00,MAIN,OxycoDONE (Immediate Release),OXYC5,004225,4.060553e+08,5mg Tablet,NaN,5-10,mg,1-2,TAB,NaN,PO/NG
289,10000826,21086876,19616931,10000826-167,167.0,2146-12-18 10:00:00,2146-12-25 00:00:00,MAIN,FoLIC Acid,FOLI1,002366,1.820508e+08,1 mg Tab,NaN,1,mg,1,TAB,1.0,PO/NG
290,10000826,21086876,19698853,NaN,NaN,2146-12-24 02:00:00,2146-12-25 00:00:00,MAIN,OxycoDONE (Immediate Release),OXYC5,004225,4.060553e+08,5mg Tablet,NaN,10,mg,2,TAB,NaN,PO


In [113]:
i = paths.index('chartevent1.csv')
CHARTEVENTS1 = datas[i].copy()
i = paths.index('chartevent2.csv')
CHARTEVENTS2 = datas[i].copy()
i = paths.index('chartevent3.csv')
CHARTEVENTS3 = datas[i].copy()
i = paths.index('chartevent4.csv')
CHARTEVENTS4 = datas[i].copy()
i = paths.index('chartevent5.csv')
CHARTEVENTS5 = datas[i].copy()
i = paths.index('chartevent6.csv')
CHARTEVENTS6 = datas[i].copy()
i = paths.index('chartevent7.csv')
CHARTEVENTS7 = datas[i].copy()
i = paths.index('chartevent8.csv')
CHARTEVENTS8 = datas[i].copy()
i = paths.index('chartevent9.csv')
CHARTEVENTS9 = datas[i].copy()
i = paths.index('chartevent10.csv')
CHARTEVENTS10 = datas[i].copy()
i = paths.index('chartevent11.csv')
CHARTEVENTS11 = datas[i].copy()
CHARTEVENTS1.head()

,Unnamed: 0,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
44059,44059,10002428,23473524,35479615,2156-05-11 14:49:00,2156-05-11 16:30:00,226512,48.4,48.4,kg,0.0
44060,44060,10002428,23473524,35479615,2156-05-11 16:00:00,2156-05-11 16:05:00,223835,40,40.0,NaN,0.0
44061,44061,10002428,23473524,35479615,2156-05-11 16:00:00,2156-05-11 16:05:00,223848,Drager,1.0,NaN,0.0
44062,44062,10002428,23473524,35479615,2156-05-11 16:00:00,2156-05-11 16:05:00,223849,CMV/ASSIST,2.0,NaN,0.0
44063,44063,10002428,23473524,35479615,2156-05-11 16:00:00,2156-05-11 16:05:00,224684,400,400.0,mL,0.0


# Features

In [44]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Initialize dataset

In [86]:
dataset = DIAGNOSES_ICD[['subject_id', 'hadm_id']].copy()
dataset.sort_values(by=['subject_id', 'hadm_id'], inplace=True)
dataset.index = range(len(dataset))
dataset

,subject_id,hadm_id
0,10000826,21086876
1,10001401,27012892
2,10001401,27012892
3,10001401,27012892
4,10001401,27060146
...,...,...
21093,19992202,20329411
21094,19997367,20617667
21095,19997367,21508795
21096,19997886,20793010


## Add gender and age to dataset

In [87]:
dataset['gender'] = pd.Series([''] * len(dataset))
dataset['age'] = pd.Series([np.nan] * len(dataset))

for i in PATIENTS.index:
    subject_id = PATIENTS['subject_id'].loc[i]
    gender = PATIENTS['gender'].loc[i]
    age = PATIENTS['anchor_age'].loc[i]
    index = dataset[dataset['subject_id'] == subject_id].index
    dataset['age'].loc[index] = age
    dataset['gender'].loc[index] = gender

dataset

,subject_id,hadm_id,gender,age
0,10000826,21086876,F,32.0
1,10001401,27012892,F,89.0
2,10001401,27012892,F,89.0
3,10001401,27012892,F,89.0
4,10001401,27060146,F,89.0
...,...,...,...,...
21093,19992202,20329411,F,80.0
21094,19997367,20617667,F,63.0
21095,19997367,21508795,F,63.0
21096,19997886,20793010,M,67.0


## Add death time to dataset

In [88]:
dataset['deathtime'] = pd.Series([''] * len(dataset))

for i in ADMISSIONS.index:
    subject_id = ADMISSIONS['subject_id'].loc[i]
    hadm_id = ADMISSIONS['hadm_id'].loc[i]
    death_time = ADMISSIONS['deathtime'].loc[i]
    index = dataset[(dataset['subject_id'] == subject_id) & (dataset['hadm_id'] == hadm_id)].index
    dataset['deathtime'].loc[index] = death_time   
    
dataset

,subject_id,hadm_id,gender,age,deathtime
0,10000826,21086876,F,32.0,NaT
1,10001401,27012892,F,89.0,NaT
2,10001401,27012892,F,89.0,NaT
3,10001401,27012892,F,89.0,NaT
4,10001401,27060146,F,89.0,NaT
...,...,...,...,...,...
21093,19992202,20329411,F,80.0,NaT
21094,19997367,20617667,F,63.0,NaT
21095,19997367,21508795,F,63.0,NaT
21096,19997886,20793010,M,67.0,NaT


## Add ICUSTAY_ID to dataset

In [89]:
dataset['icustay_id'] = pd.Series([''] * len(dataset))
dataset['intime'] = pd.Series([''] * len(dataset))
dataset['outtime'] = pd.Series([''] * len(dataset))

# use intime - dob to calculate age
for i in ICUSTAYS.index:
    subject_id = ICUSTAYS['subject_id'].loc[i]
    hadm_id = ICUSTAYS['hadm_id'].loc[i]
    icustay_id = ICUSTAYS['stay_id'].loc[i]
    intime = ICUSTAYS['intime'].loc[i]
    outtime = ICUSTAYS['outtime'].loc[i]
    index = dataset[(dataset['subject_id'] == subject_id) & (dataset['hadm_id'] == hadm_id)].index
    if dataset['icustay_id'].loc[index[0]] == '':
        dataset['icustay_id'].loc[index[0]] = icustay_id
        dataset['intime'].loc[index[0]] = intime
        dataset['outtime'].loc[index[0]] = outtime
    else:
        dataset.loc[len(dataset)] = [dataset['subject_id'].loc[index[0]],
                                     dataset['hadm_id'].loc[index[0]],
                                     dataset['gender'].loc[index[0]],
                                     dataset['age'].loc[index[0]],
                                     dataset['deathtime'].loc[index[0]],
                                     icustay_id,
                                     intime,
                                     outtime]

dataset.sort_values(by=['subject_id', 'intime'])
dataset = dataset[['subject_id', 'hadm_id', 'icustay_id', 'intime', 'outtime', 'gender', 'age', 'deathtime']]
dataset

,subject_id,hadm_id,icustay_id,intime,outtime,gender,age,deathtime
0,10000826,21086876,,,,F,32.0,NaT
1,10001401,27012892,,,,F,89.0,NaT
2,10001401,27012892,,,,F,89.0,NaT
3,10001401,27012892,,,,F,89.0,NaT
4,10001401,27060146,,,,F,89.0,NaT
...,...,...,...,...,...,...,...,...
23263,19970491,25338284,38969862,2129-09-15 18:36:09,2129-09-17 22:08:50,M,55.0,NaT
23264,19983257,21588174,31169972,2166-06-19 09:53:03,2166-06-20 02:41:16,M,64.0,2166-06-19 21:15:00
23265,19986715,28333468,39855146,2150-11-29 03:59:00,2150-12-09 17:56:59,F,20.0,NaT
23266,19988077,28414691,32614619,2140-05-10 15:44:42,2140-05-11 17:40:58,M,53.0,NaT


## Add re-admission to dataset

In [90]:
dataset['re_admission'] = pd.Series([''] * len(dataset))

dataset['re_admission'].loc[0] = 0
for i in dataset.index[1:]:
    if dataset['subject_id'].loc[i] == dataset['subject_id'].loc[i - 1]:
        dataset['re_admission'].loc[i] = 1
    else:
        dataset['re_admission'].loc[i] = 0

dataset

,subject_id,hadm_id,icustay_id,intime,outtime,gender,age,deathtime,re_admission
0,10000826,21086876,,,,F,32.0,NaT,0
1,10001401,27012892,,,,F,89.0,NaT,0
2,10001401,27012892,,,,F,89.0,NaT,1
3,10001401,27012892,,,,F,89.0,NaT,1
4,10001401,27060146,,,,F,89.0,NaT,1
...,...,...,...,...,...,...,...,...,...
23263,19970491,25338284,38969862,2129-09-15 18:36:09,2129-09-17 22:08:50,M,55.0,NaT,1
23264,19983257,21588174,31169972,2166-06-19 09:53:03,2166-06-20 02:41:16,M,64.0,2166-06-19 21:15:00,0
23265,19986715,28333468,39855146,2150-11-29 03:59:00,2150-12-09 17:56:59,F,20.0,NaT,0
23266,19988077,28414691,32614619,2140-05-10 15:44:42,2140-05-11 17:40:58,M,53.0,NaT,0


# Split time

In [98]:
hour_period = '10:00:00'

In [101]:
for hour in range(1, 4):
    hour_period = f'0{hour}:00:00'
    
    index = 0
    split = pd.DataFrame(columns = dataset.columns)
    for i in tqdm(dataset.index):
        intime, outtime = dataset['intime'].loc[i], dataset['outtime'].loc[i]
        if intime == '' or 'outtime' == '':
            continue
        while outtime - intime > pd.Timedelta(hour_period):
            #['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']
            split.loc[index] = [
                dataset['subject_id'].loc[i],
                dataset['hadm_id'].loc[i],
                dataset['icustay_id'].loc[i],
                intime,
                intime + pd.Timedelta(hour_period),
                dataset['gender'].loc[i],
                dataset['age'].loc[i],
                dataset['deathtime'].loc[i],
                dataset['re_admission'].loc[i],
            ]
            intime += pd.Timedelta(hour_period)
            index += 1

        split.loc[index] = [
            dataset['subject_id'].loc[i],
            dataset['hadm_id'].loc[i],
            dataset['icustay_id'].loc[i],
            intime,
            outtime,
            dataset['gender'].loc[i],
            dataset['age'].loc[i],
            dataset['deathtime'].loc[i],
            dataset['re_admission'].loc[i],
        ]

    split.to_csv(f'../data/mimiciv/dataset_split_{hour}_hour.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23268/23268 [49:09<00:00,  7.89it/s]


In [100]:
split.to_csv('../data/mimiciv/dataset_split_10_hour.csv')

# Features

In [102]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Load dataset after split time

## Load variable range

In [109]:
var_range = pd.read_csv('../data/mimiciv/variable_range.csv')
var_range.index = var_range.iloc[:, 0]
var_range = var_range.iloc[:, 1:]
var_range['GROUP_ID'] = var_range['GROUP_ID'].apply(lambda x : [int(s) for s in x.split(',')] if type(x) == type('') else x)
var_range

,OUTLIER LOW,VALID LOW,IMPUTE,VALID HIGH,OUTLIER HIGH,GROUP_ID
Unnamed: 0,,,,,,
Albumin,0.0,0.60,3.10,6.0,60.0,"[50862, 772, 1521, 227456]"
Arterial pH,NaN,NaN,NaN,NaN,NaN,"[780, 1126, 223830, 50820]"
Calcium,NaN,NaN,NaN,NaN,NaN,"[225625, 50893, 786, 1522]"
Glucose,0.0,33.00,128.00,2000.0,2200.0,"[220621, 225664, 50809, 811, 807, 50931, 22653..."
Hemoglobin,0.0,0.00,10.20,25.0,30.0,"[220228, 51222, 50811, 814]"
Magnesium,0.0,0.00,2.00,20.0,22.0,"[220635, 50960, 1532, 821]"
PTT,0.0,18.80,34.30,150.0,150.0,"[227466, 825, 1533, 51275]"
Postassium,0.0,0.00,4.10,12.0,15.0,"[829, 50971, 1535, 227464, 50822]"
SGPT,NaN,NaN,NaN,NaN,NaN,[3802]


## Merge features to dataset

In [ ]:
def merge(dataset, feature, EVENTS, var_range, have_icustay_id=True):
    # find all feature records in EVENTS
    event = EVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in EVENTS['itemid']]]

    for i in range(len(event)):
        subject_id = event.iloc[i]['subject_id']
        hadm_id = event.iloc[i]['hadm_id']
        if have_icustay_id:
            icustay_id = event.iloc[i]['stay_id']
        time = event.iloc[i]['charttime']
        
        # detect outlier value, if out of outlier range then set to missing value
        value = event.iloc[i]['valuenum']
        if var_range.loc[feature].notnull()[0]:
            if value > var_range.loc[feature]['OUTLIER HIGH']:
                value = np.nan
            elif value > var_range.loc[feature]['VALID HIGH']:
                value = var_range.loc[feature]['VALID HIGH']
            elif value < var_range.loc[feature]['OUTLIER LOW']:
                value = np.nan
            elif value < var_range.loc[feature]['VALID LOW']:
                value = var_range.loc[feature]['VALID LOW']
        
        if have_icustay_id:
            index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id][dataset['icustay_id'] == icustay_id]).index
        else:
            index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id]).index
        
        for idx in index:
            if time - dataset.iloc[idx]['starttime'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['endtime'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1
                
                if have_icustay_id:
                    break

for hour in tqdm(range(10, 0, -1)):
    path = f'../data/mimiciv/dataset_split_{hour}_hour.csv'

    dataset = pd.read_csv(path)

    # change time column data's type to timestamp type
    dataset['intime'] = dataset['intime'].apply(lambda x : pd.Timestamp(x))
    dataset['outtime'] = dataset['outtime'].apply(lambda x : pd.Timestamp(x))
    dataset['deathtime'] = dataset['deathtime'].apply(lambda x : pd.Timestamp(x))
    dataset.columns = ['Unnamed:0', 'subject_id', 'hadm_id', 'icustay_id', 'starttime', 'endtime', 'gender',
                       'age', 'deathtime', 're_admission']
    dataset = dataset.iloc[:, 1:]
    # dataset.head()                    


    for feature in tqdm(var_range.index):
        # count records in same period
        dataset['records'] = pd.Series([0] * len(dataset), dtype = np.int64)

        # add new feature column
        dataset[feature] = pd.Series([np.nan] * len(dataset), dtype = np.float64)

        # add data to new feature column with corresponding subject_id, hadm_id, icustay_id and time
        merge(dataset, feature, LABEVENTS, var_range, have_icustay_id = False)
        merge(dataset, feature, OUTPUTEVENTS, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS1, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS2, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS3, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS4, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS5, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS6, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS7, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS8, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS9, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS10, var_range, have_icustay_id = True)
        merge(dataset, feature, CHARTEVENTS11, var_range, have_icustay_id = True)

    dataset.to_csv(f'../data/mimiciv/dataset_split_{hour}_hour_after_merge.csv')

  0%|                                                                                                                                                                               | 0/35 [00:00<?, ?it/s]/tmp/ipykernel_32732/2160453800.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id]).index
/tmp/ipykernel_32732/2160453800.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id][dataset['icustay_id'] == icustay_id]).index

  3%|████▋                                                                                                                                                             | 1/35 [54:28<30:52:15, 3268.68s/it]/tmp/ipykernel_32732/2160453800.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[data

 43%|███████████████████████████████████████████████████████████████████▋                                                                                          | 15/35 [16:13:34<19:21:48, 3485.42s/it]/tmp/ipykernel_32732/2160453800.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id]).index
/tmp/ipykernel_32732/2160453800.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['subject_id'] == subject_id][dataset['hadm_id'] == hadm_id][dataset['icustay_id'] == icustay_id]).index

 46%|████████████████████████████████████████████████████████████████████████▏                                                                                     | 16/35 [17:26:55<19:51:04, 3761.30s/it]/tmp/ipykernel_32732/2160453800.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[data